# Miscellanous useful snippets for the Open Data Cube

In [1]:
# Supress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import datacube
dc = datacube.Datacube(app='my_app')

## List available products

In [3]:
dc.list_products()

,name,description,label,product_type,format,instrument,creation_time,platform,time,lat,lon,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,
4,ls5_usgs_sr_scene,Landsat 5 USGS Collection 1 Level2 Surface Ref...,None,LEVEL2_USGS,GeoTIFF,TM,None,LANDSAT_5,None,None,None,NaN,NaN,NaN,NaN
3,ls7_usgs_sr_scene,Landsat 7 USGS Collection 1 Level2 Surface Ref...,None,LEVEL2_USGS,GeoTIFF,ETM,None,LANDSAT_7,None,None,None,NaN,NaN,NaN,NaN
5,ls8_usgs_fc_scene,Landsat 8 Fractional Cover based on USGS Level...,None,fractional_cover,GeoTiff,OLI_TIRS,None,LANDSAT_8,None,None,None,NaN,NaN,NaN,NaN
2,ls8_usgs_sr_scene,Landsat 8 USGS Collection 1 Higher Level SR sc...,None,LEVEL2_USGS,GeoTIFF,OLI_TIRS,None,LANDSAT_8,None,None,None,NaN,NaN,NaN,NaN
1,ls8_usgs_wofs_scene,Historic Flood Mapping Water Observations from...,None,wofs,GeoTIFF,None,None,None,None,None,None,NaN,NaN,NaN,NaN


## List measurements for each product

In [4]:
dc.list_measurements()

aliases  \
product             measurement                                              
ls5_usgs_sr_scene   blue                                [band_1, sr_band1]   
                    green                               [band_2, sr_band2]   
                    red                                 [band_3, sr_band3]   
                    nir                                 [band_4, sr_band4]   
                    swir1                               [band_5, sr_band5]   
                    swir2                               [band_7, sr_band7]   
                    sr_atmos_opacity                            [atmos_op]   
                    pixel_qa                                    [pixel_qa]   
                    radsat_qa                                  [radsat_qa]   
                    sr_cloud_qa                                 [pixel_qa]   
ls7_usgs_sr_scene   blue                                [band_1, sr_band1]   
                    green                               [band_2, sr_band2]   
                    red                                 [band_3, sr_band3]   
                    nir                                 [band_4, sr_band4]   
                    swir1                               [band_5, sr_band5]   
                    swir2                               [band_7, sr_band7]   
                    sr_atmos_opacity                            [atmos_op]   
                    pixel_qa                                    [pixel_qa]   
                    radsat_qa                                  [radsat_qa]   
                    sr_cloud_qa                                 [cloud_qa]   
ls8_usgs_fc_scene   BS                                              [bare]   
                    PV                                         [green_veg]   
                    NPV                                         [dead_veg]   
                    UE                                               [err]   
ls8_usgs_sr_scene   coastal_aerosol                     [band_1, sr_band1]   
                    blue                                [band_2, sr_band2]   
                    green                               [band_3, sr_band3]   
                    red                                 [band_4, sr_band4]   
                    nir                                 [band_5, sr_band5]   
                    swir1                               [band_6, sr_band6]   
                    swir2                               [band_7, sr_band7]   
                    pixel_qa                                    [pixel_qa]   
                    sr_aerosol        [sr_aerosol_qa, aerosol_qa, aerosol]   
                    radsat_qa                                  [radsat_qa]   
ls8_usgs_wofs_scene water                                              NaN   

                                       dtype  \
product             measurement                
ls5_usgs_sr_scene   blue               int16   
                    green              int16   
                    red                int16   
                    nir                int16   
                    swir1              int16   
                    swir2              int16   
                    sr_atmos_opacity   uint8   
                    pixel_qa          uint16   
                    radsat_qa          uint8   
                    sr_cloud_qa       uint16   
ls7_usgs_sr_scene   blue               int16   
                    green              int16   
                    red                int16   
                    nir                int16   
                    swir1              int16   
                    swir2              int16   
                    sr_atmos_opacity   uint8   
                    pixel_qa          uint16   
                    radsat_qa          uint8   
                    sr_cloud_qa        uint8   
ls8_usgs_fc_scene   BS                 int16   
                    PV                 int16   
                    NPV     

## Count the number of datasets for a given product

In [5]:
len(dc.find_datasets(product='ls8_usgs_sr_scene'))

1927

# Dask distributed

In [6]:
import dask
from dask.distributed import Client
client = Client('dask-scheduler.dask.svc.cluster.local:8786')
client

Client Scheduler: tcp://dask-scheduler.dask.svc.cluster.local:8786 Dashboard: http://dask-scheduler.dask.svc.cluster.local:8787/status,Cluster Workers: 3 Cores: 24 Memory: 101.19 GB


## Configure GDAL for cloud access on every Dask worker

In [7]:
def worker_setup_auto():
    from datacube.utils.rio import set_default_rio_config, activate_from_config
    
    # these settings will be applied in every worker thread
    set_default_rio_config(aws={'aws_unsigned': True},
                           cloud_defaults=True)
    
    # Force activation in the main thread
    # - Really just to test that configuration works
    # - Every worker thread will automatically run this again
    return activate_from_config()

In [8]:
# Runs once on every worker process, not per worker thread!
client.register_worker_callbacks(setup=worker_setup_auto)

{'tcp://10.244.1.8:34729': {'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
  'AWS_NO_SIGN_REQUEST': 'YES'},
 'tcp://10.244.2.7:42243': {'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
  'AWS_NO_SIGN_REQUEST': 'YES'},
 'tcp://10.244.3.8:43257': {'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
  'AWS_NO_SIGN_REQUEST': 'YES'}}

In [9]:
client.restart()

Client Scheduler: tcp://dask-scheduler.dask.svc.cluster.local:8786 Dashboard: http://dask-scheduler.dask.svc.cluster.local:8787/status,Cluster Workers: 3 Cores: 24 Memory: 101.19 GB


# Where to go from here?
For more complex scenarios, have a look at this set of [Jupyter Notebooks](https://github.com/opendatacube/dea-proto/tree/master/notebooks).